In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import dvu
import seaborn as sns
import os
import pandas as pd
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
import joblib
dvu.set_style()

# results_dir = '../results_mar25'
results_dir = '../results'
experiment_filename = '../experiments/01_fit_encoding.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
# r = imodelsx.process_results.fill_missing_args_with_default(
    # r, experiment_filename)

100%|██████████| 4/4 [00:00<00:00, 282.64it/s]
/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/pkg_resources/__init__.py:2350: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)
/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/lightning/fabric/__init__.py:41: Deprecated call

experiment varied these params: ['feature_space', 'pc_components']


In [7]:
mets = [c for c in r.columns if 'corrs_test_' in c] + \
    [c for c in r.columns if 'corrs_tune_' in c]
mets = [met for met in mets if not 'pc' in met]
d = r.groupby(cols_varied)[mets].mean()
(
    d
    .sort_values(by='corrs_test_mean', ascending=False)
    .rename(columns=lambda x: x.replace('_', ' ').replace('corrs', ''))
    .style
    .background_gradient(cmap='viridis', axis=0)
    .format(precision=3)
)

# Analyze

In [ ]:
mets

In [ ]:
row = r.iloc[0]
model_params = joblib.load(join(row.save_dir_unique, 'model_params.pkl'))

In [ ]:
model_params['alphas_best'].shape